Script per capire  il funzionamento del calcolo delle similarità con **jarowinkler**.    

## jarowinkler

In sintesi nel seguente script si trova conferma che:

- Il candidate pairs è case sensitive.

-  Jarowinkler è case sensitive.

- **jarowinkler** non guarda solo alle lettere presenti ma anche alla loro disposizione.

In [26]:
import pandas as pd 
import recordlinkage
def similarity_winkler(dataset):
   dataset["indice"]= dataset.index

   dataset = dataset.set_index("indice")  # Impostare l'indice richiesto da recordlinkage

   # Step 2: Aggiungere colonna per il blocking (prima lettera)
   dataset["first_letter"] = dataset["titolo"].str[0]

   # Step 3: Creare l'indice con blocking sulla prima lettera
   indexer = recordlinkage.Index()
   indexer.block("first_letter")
   candidate_links = indexer.index(dataset)

   print(f"Coppie candidate: {len(candidate_links)}")

   # Step 4: Confronto tra i titoli
   compare = recordlinkage.Compare()
   compare.string('titolo', 'titolo', method='jarowinkler', label='name_similarity')

   features = compare.compute(candidate_links, dataset)

   # Mostrare le coppie con similarità
   features.reset_index(inplace=True)
   features = pd.merge(features,dataset,left_on="indice_1",right_on="indice")
   features = pd.merge(features,dataset,left_on="indice_2",right_on="indice",suffixes=("_1","_2"))
   features.drop(columns=["first_letter_1","first_letter_2"],inplace=True)
   return features

In [27]:
prova = pd.DataFrame({
    "titolo": ["Roma",
                "Ramo",
                "roma",
                "ramo"
                ]
})

similarity_winkler(prova.copy())

Coppie candidate: 2


,indice_1,indice_2,name_similarity,titolo_1,titolo_2
0,1,0,0.666667,Ramo,Roma
1,3,2,0.666667,ramo,roma


Roma e roma non prese in considerazione perchè il **candidate pairs** di default è case sensitive.

In [28]:
# Step 1: Creare il DataFrame
prova = pd.DataFrame({
    "titolo": ["Roma",
                "Ramo",
                "Roma",
                "Ramo"
                ]
})

similarity_winkler(prova.copy())

Coppie candidate: 6


,indice_1,indice_2,name_similarity,titolo_1,titolo_2
0,1,0,0.666667,Ramo,Roma
1,2,0,1.000000,Roma,Roma
2,2,1,0.666667,Roma,Ramo
3,3,0,0.666667,Ramo,Roma
4,3,1,1.000000,Ramo,Ramo
5,3,2,0.666667,Ramo,Roma


Ora Roma e Roma sono prese in considerazione, così come Roma e Ramo, conferma che il **candidate pairs** case sensitive.

In [29]:
prova = pd.DataFrame({
    "titolo": ["Roma",
                "RoMa",
               "Roma"
                ]
})

similarity_winkler(prova.copy())

Coppie candidate: 3


,indice_1,indice_2,name_similarity,titolo_1,titolo_2
0,1,0,0.866667,RoMa,Roma
1,2,0,1.000000,Roma,Roma
2,2,1,0.866667,Roma,RoMa


Conferma che la similarità di **jarowinkler** è case sensitive. Roma risulta diverso da RoMa.

In [30]:
prova = pd.DataFrame({
    "titolo": ["Roma",
               "Ramo"
                ]
})

similarity_winkler(prova.copy())

Coppie candidate: 1


,indice_1,indice_2,name_similarity,titolo_1,titolo_2
0,1,0,0.666667,Ramo,Roma


**jarowinkler** non guarda solo alle lettere presenti ma anche alla loro disposizione. Ramo e Roma hanno le stesse lettere ma valori di similarità diversi.

# jellyfish

### Applicare **jellyfish** a due stringhe

Applicare **jellyfish**, ovvero calcolare direttamente tra due stringhe la loro similarity.    
In questo caso si applica la **jaro_winkler_similarity**.

In [38]:
print(f'Similarità "Charles Leclerc","Charles Bass": {jellyfish.jaro_winkler_similarity("Charles Leclerc","Charles Bass")}')

print(f'Similarità "Charles","Charles": {jellyfish.jaro_winkler_similarity("Charles","Charles")}')

print(f'Similarità "Charles","charles": {jellyfish.jaro_winkler_similarity("Charles","charles")}')

Similarità "Charles Leclerc","Charles Bass": 0.8400000000000001
Similarità "Charles","Charles": 1.0
Similarità "Charles","charles": 0.9047619047619048


Aiuta molto a capire come funziona **jarowinkler**, si nota infatti l'importanza che attribuisce alle minuscole o alle maiuscole. 

### Applicare **jellyfish** ad un dataset.

In [31]:
import jellyfish
path_dati = "/Users/mattia/Desktop/Università/Data Science in Python/_Progetti/MatchAnalysis_Imputation/Progetto/Results/"
df = pd.read_csv(f"{path_dati}/matches_titolo_095.csv",index_col=0)


df['name_similarity'] = df.apply(
    lambda row: jellyfish.jaro_winkler_similarity(
        row['titolo_booking'], row['titolo_agoda']
    ),
    axis=1
)

# Conta quelli con similarity > 0.95
high_sim_name = (df['name_similarity'] > 0.95).sum()
print(f"Match con nomi molto simili (Jaro-Winkler > 0.95): {high_sim_name} su {len(df)} totali")

Match con nomi molto simili (Jaro-Winkler > 0.95): 247 su 275 totali
